In [1]:
import plotly
import pygal
import matplotlib.pyplot as plt
import numpy as np
from currency_converter import CurrencyConverter
from IPython.display import SVG, display
c = CurrencyConverter()

## Class defs

In [40]:
class Storage:
    def __init__(self, name, location, currency):
        self.name = name
        self.currency = currency
        self.location = location
    def __repr__(self):          
        string=f"\nName       : {self.name}\nLocation   : {self.location}"
        return string
    def __str__(self):
        return self.__repr__()
    
    # setters
    def set_storage(self, storage):
        self.storage = storage *1024
        return self
    def set_manipuation_requests(self, manipulation_requests):
        self.manipulation_requests = manipulation_requests
    def set_retrieval_requests(self, retrieval_requests):
        self.retrieval_requests = retrieval_requests
    def set_data_retrieval(self, data_retrieval):
        self.data_retrieval = data_retrieval
    def set_outbound_transfer(self, outbound_transfer):
        self.outbound_transfer = outbound_transfer
    
class Cloud(Storage):
    def set_costs(self,
                  storage_tiers, 
                  storage_costs, 
                  manipulation_request_cost,
                  retrieval_request_cost,
                  retrieval_costs,
                  outbound_tiers,
                  outbound_costs
                 ):
        self.storage_tiers = storage_tiers
        self.storage_costs = storage_costs
        
        #requests
        self.manipulation_request_cost = manipulation_request_cost
        self.retrieval_request_cost = retrieval_request_cost
        #retrieval
        self.retrieval_cost = retrieval_costs
        #outbound
        self.outbound_tiers = outbound_tiers
        self.outbound_costs = outbound_costs
    
    
    def calc_storage_cost(self):
        tiers = self.storage_tiers
        costs = self.storage_costs
        storage = self.storage
        total = 0
        for tier, cost in zip(tiers, costs):
            if storage > tier:
                total += cost*tier
                storage -= tier
            else:
                total += cost*storage
                break
        return total
    def calc_retrieval_cost(self):
        return self.retieval_cost*self.data_retrieval
    def calc_manipulation_request_cost(self):
        return self.manipulation_request_cost*self.manipulation_requests
    def calc_retrieval_request_cost(self):
        return self.retrieval_request_cost*self.retrieval_requests
    def calc_retrieval_cost(self):
        return self.data_retrieval*self.retrieval_cost
    def calc_outbound_cost(self):
        tiers = self.outbound_tiers
        costs = self.outbound_costs
        outbound = self.outbound_transfer
        total = 0
        for tier, cost in zip(tiers, costs):
            if outbound > tier:
                total += cost*tier
                outbound -= tier
            else:
                total += cost*outbound
                break
        return total
    def monthly_cost(self):
        cost = sum((
                    Cloud.calc_storage_cost(self),
                    Cloud.calc_manipulation_request_cost(self),
                    Cloud.calc_retrieval_request_cost(self),
                    Cloud.calc_retrieval_cost(self),
                    Cloud.calc_outbound_cost(self)
                ))
        if self.currency is "CAD":
            return cost
        else:
            return c.convert(cost, self.currency, 'CAD')
    def yearly_cost(self):
        return Cloud.monthly_cost(self)*12
    
class Sync(Storage):
    def set_costs(self, storage_cost, per_user=False):
        self.storage_cost = storage_cost
        self.per_user = per_user
    def set_users(self, users):
        self.users = users
    def monthly_cost(self):
        if self.per_user:
            return self.storage_cost/12*self.users*self.storage/1024*1000
        return self.storage_cost/12*self.storage/1024*1000
    def yearly_cost(self):
        return Sync.monthly_cost(self)*12

## Defining cloud and sync options

In [26]:
azure_archive_LRS_east = Cloud('Azure Archive LRS', 'Canada East', 'CAD')
azure_archive_LRS_east.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0031,  0.0031,   0.0031], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.68/10000,
    retrieval_costs = 0.0308,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [27]:
azure_archive_GRS_east = Cloud('Azure Archive GRS', 'Canada East', 'CAD')
azure_archive_GRS_east.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0060,  0.0060,   0.0060], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.68/10000,
    retrieval_costs = 0.0308,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [28]:
azure_archive_LRS_central = Cloud('Azure Archive LRS', 'Canada Central', 'CAD')
azure_archive_LRS_central.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0029,  0.0029,   0.0029], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.04/10000,
    retrieval_costs = 0.0282,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [29]:
azure_archive_GRS_central = Cloud('Azure Archive GRS', 'Canada Central', 'CAD')
azure_archive_GRS_central.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0060,  0.0060,   0.0060], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.04/10000,
    retrieval_costs = 0.0282,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [30]:
amazon_glacier_canada = Cloud('Amazon S3 Glacier BULK', 'Canada Central', 'USD')
amazon_glacier_canada.set_costs(
    storage_tiers = [1e31], 
    storage_costs = [0.0045], 
    manipulation_request_cost = 0.055/1000,
    retrieval_request_cost = 0.0275/1000,
    retrieval_costs = 0.00275,
    outbound_tiers = [1, 10*1024, 50*1024, 150*1024, 1e31],
    outbound_costs = [0, 0.09,    0.085,   0.07,    0.05]
)

In [31]:
google_coldline_canada = Cloud('Google Cloud Coldline', 'Montreal', 'CAD')
google_coldline_canada.set_costs(
    storage_tiers = [1e31], 
    storage_costs = [0.007], 
    manipulation_request_cost = 0.1/10000,
    retrieval_request_cost = 0.05/10000,
    retrieval_costs = 0.05,
    outbound_tiers = [1024, 10*1024, 1e31],
    outbound_costs = [0.12, 0.11,    0.08]
)

In [32]:
teamshare = Sync('Teamshare', 'UBC', 'CAD')
teamshare.set_costs(0.15)

### Plotting func def

In [33]:
def storage_cost(choices, attributes, filename, annual=False):
    if annual:
        chart = pygal.XY(xrange=(0,501), title='Archival Annual Comparison', x_title='Storage(TB)', y_title='Cost per Year (CAD)') 
    else:
        chart = pygal.XY(xrange=(0,501), title='Archival Monthly Comparison', x_title='Storage(TB)', y_title='Cost per Month (CAD)')
    x = np.arange(0, 500, 0.25)
    for choice in choices:
        choice.set_manipuation_requests(attributes['manipulation_requests'])
        choice.set_retrieval_requests(attributes['retrieval_requests'])
        choice.set_data_retrieval(attributes['data_retrieval'])
        choice.set_outbound_transfer(attributes['outbound_trasfer'])
        if annual:
            y = [(i, choice.set_storage(i).yearly_cost()) for i in x]
        else:
            y = [(i, choice.set_storage(i).monthly_cost()) for i in x]
        label = choice.name + " " + choice.location
        chart.add(label, y)
    chart.render_to_file(filename) 

## Making plots

In [34]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'cloud_annual.svg', True)

In [35]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'cloud_monthly.svg', False)

In [36]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada,
           teamshare]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'storage_monthly.svg', False)

In [39]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada,
           teamshare]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'storage_annual.svg', True)